Import necessary Library

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')



In [2]:
df = pd.read_csv(r'D:\My Prog\Naresh_i _Technology\Resume Project\Customer Churn Analysis\Churn_Modelling.csv')

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,delhi,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,bangalore,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,delhi,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,delhi,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,bangalore,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [5]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [6]:
for col in df.columns:
    print(df[col].value_counts())

RowNumber
1        1
6671     1
6664     1
6665     1
6666     1
        ..
3334     1
3335     1
3336     1
3337     1
10000    1
Name: count, Length: 10000, dtype: int64
CustomerId
15634602    1
15667932    1
15766185    1
15667632    1
15599024    1
           ..
15599078    1
15702300    1
15660735    1
15671390    1
15628319    1
Name: count, Length: 10000, dtype: int64
Surname
Smith       32
Scott       29
Martin      29
Walker      28
Brown       26
            ..
Izmailov     1
Bold         1
Bonham       1
Poninski     1
Burbidge     1
Name: count, Length: 2932, dtype: int64
CreditScore
850    233
678     63
655     54
705     53
667     53
      ... 
404      1
351      1
365      1
417      1
419      1
Name: count, Length: 460, dtype: int64
Geography
delhi        5014
mumbai       2509
bangalore    2477
Name: count, dtype: int64
Gender
Male      5457
Female    4543
Name: count, dtype: int64
Age
37    478
38    477
35    474
36    456
34    447
     ... 
92      2
82      1


Defining input var & target var

In [7]:
x = df.iloc[:,3:-1].values
y = df.iloc[:,-1].values

In [8]:
x

array([[619, 'delhi', 'Female', ..., 1, 1, 101348.88],
       [608, 'bangalore', 'Female', ..., 0, 1, 112542.58],
       [502, 'delhi', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'delhi', 'Female', ..., 0, 1, 42085.58],
       [772, 'mumbai', 'Male', ..., 1, 0, 92888.52],
       [792, 'delhi', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [9]:
x

array([[619, 'delhi', 'Female', ..., 1, 1, 101348.88],
       [608, 'bangalore', 'Female', ..., 0, 1, 112542.58],
       [502, 'delhi', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'delhi', 'Female', ..., 0, 1, 42085.58],
       [772, 'mumbai', 'Male', ..., 1, 0, 92888.52],
       [792, 'delhi', 'Female', ..., 1, 0, 38190.78]], dtype=object)

### Encoding categorical var

One hot encoding the Geography column

In [10]:
# from sklearn.preprocessing import OneHotEncoder
# encoder = OneHotEncoder()
# # x['Geography']=encoder.fit_transform(x[['Geography']])
# one_hot_encoded = encoder.fit_transform(x[['Geography']])

# # Convert the sparse matrix to a dense array
# one_hot_array = one_hot_encoded.toarray()

# # Assign the dense array to the DataFrame column
# x['Geography'] = one_hot_array
# # 
# # one_hot_encoded = encoder.transform(df[['Category']]).toarray()

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct_col = ColumnTransformer(transformers = [('encoder',OneHotEncoder(),[1])],remainder = 'passthrough')
x = np.array(ct_col.fit_transform(x))

Label encoding the age column

In [11]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:,2]= le.fit_transform(x[:,2])

One hot encoding the 'geography' column

In [12]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
x = np.array(ct.fit_transform(x))

Splitting data into train and test set

In [13]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [14]:
xtrain[0]

array([1.0, 0.0, 0.0, 667, 0, 34, 5, 0.0, 2, 1, 0, 163830.64],
      dtype=object)

In [15]:
ytrain

array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

Let's apply all the classification algorithm.
- Logistic Regression
- Naive Bayes
- Decision Trees
- Random Forest
- Support Vector Machines (SVM)
- k-Nearest Neighbors (k-NN)
- Gradient Boosting (e.g., XGBoost, LightGBM, CatBoost)

# Let's apply Logistic Regression

In [16]:
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression()
logit.fit(xtrain,ytrain)

LogisticRegression()

In [17]:
# Prediction
logit_pred = logit.predict(xtest)

In [18]:
logit_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

#### Let's Calculate all other necessary information

In [19]:
# Accuracy
logit_ac = accuracy_score(ytest,logit_pred)
print('So the accuracy of the model is:',logit_ac)

# Bias
logit_bias = logit.score(xtrain,ytrain)
print('The bias of the model is:',logit_bias)
logit_variance = logit.score(xtest,ytest)
print('The variance of the model is:',logit_variance)

# Accuracy table
logit_cr = classification_report(ytest,logit_pred)
print('The classification report for logistic regression is:\n',logit_cr)
# table = logit.accuracy_score(ytest,logit_pred)

So the accuracy of the model is: 0.789
The bias of the model is: 0.789875
The variance of the model is: 0.789
The classification report for logistic regression is:
               precision    recall  f1-score   support

           0       0.80      0.97      0.88      1595
           1       0.37      0.06      0.11       405

    accuracy                           0.79      2000
   macro avg       0.59      0.52      0.49      2000
weighted avg       0.72      0.79      0.72      2000



# Let's apply Naive Bayes using GridSearch

First on Multinomial Naive Bayes

In [20]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()

In [21]:
# parameter grid
param_grid = {
    'alpha': [0.1, 0.5, 1.0],  # Smoothing parameter (Laplace smoothing)
    'fit_prior': [True, False]  # Whether to learn class prior probabilities
}

In [22]:
#Applying gridsearch cv
grid_search = GridSearchCV(estimator = mnb,param_grid = param_grid,cv = 5)

In [23]:
grid_search.fit(xtrain,ytrain)

GridSearchCV(cv=5, estimator=MultinomialNB(),
             param_grid={'alpha': [0.1, 0.5, 1.0], 'fit_prior': [True, False]})

In [24]:
best = grid_search.best_params_
best

{'alpha': 0.1, 'fit_prior': True}

In [25]:
mnb_pred = grid_search.predict(xtest)
mnb_pred

array([0, 0, 1, ..., 1, 1, 0], dtype=int64)

In [26]:
mnb_ac = accuracy_score(ytest,mnb_pred)
mnb_ac # without cv ac= 0.544

0.544

In [27]:
cr = classification_report(ytest,mnb_pred)
cr

'              precision    recall  f1-score   support\n\n           0       0.83      0.54      0.65      1595\n           1       0.23      0.55      0.33       405\n\n    accuracy                           0.54      2000\n   macro avg       0.53      0.55      0.49      2000\nweighted avg       0.71      0.54      0.59      2000\n'

In [28]:
bias = grid_search.score(xtrain,ytrain)
bias # without cv 0.551875

0.551875

In [29]:
variance = grid_search.score(xtest,ytest)
variance # without cv 0.544

0.544

Second on Gaussian Naive bayes

In [30]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(xtrain,ytrain)

GaussianNB()

In [31]:
gnb_pred = gnb.predict(xtest)
gnb_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [32]:
gnb_ac = accuracy_score(ytest,gnb_pred)
gnb_ac

0.7845

In [33]:
bias = gnb.score(xtrain,ytrain)
bias

0.785375

In [34]:
variance = gnb.score(xtest,ytest)
variance

0.7845

In [35]:
cr = classification_report(ytest,gnb_pred)
cr

'              precision    recall  f1-score   support\n\n           0       0.81      0.96      0.88      1595\n           1       0.36      0.09      0.14       405\n\n    accuracy                           0.78      2000\n   macro avg       0.59      0.52      0.51      2000\nweighted avg       0.72      0.78      0.73      2000\n'

Let's go for Bernouli naive bayes algorithm

In [36]:
from sklearn.naive_bayes import BernoulliNB
bnb = BernoulliNB()
bnb.fit(xtrain,ytrain)

BernoulliNB()

In [37]:
bnb_pred = bnb.predict(xtest)
bnb_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [38]:
bnb_ac = accuracy_score(ytest,bnb_pred)
bnb_ac

0.786

In [39]:
bias = bnb.score(xtrain,ytrain)
bias

0.790375

In [40]:
variance = bnb.score(xtest,ytest)
variance

0.786

# Let's apply Decision Tree using GridSearch

In [41]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()

In [42]:
param_grid = {
    'criterion': ['gini', 'entropy'],  # Splitting criterion
    'splitter': ['best', 'random'],  # Strategy for choosing splits
    'max_depth': [None, 10, 20, 30, 40],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum samples required to split a node
    'min_samples_leaf': [1, 2, 4],  # Minimum samples required at a leaf node
    'max_features': [None, 'sqrt', 'log2'],  # Number of features to consider for best split
}

In [43]:
model = GridSearchCV(estimator = dtc, param_grid = param_grid,cv = 5)
model.fit(xtrain,ytrain)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 10, 20, 30, 40],
                         'max_features': [None, 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'splitter': ['best', 'random']})

In [44]:
best_params = model.best_params_
best_params
'''
With cv the parameters are
{'criterion': 'entropy',
 'max_depth': 10,
 'max_features': None,
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'splitter': 'random'}
'''

"\nWith cv the parameters are\n{'criterion': 'entropy',\n 'max_depth': 10,\n 'max_features': None,\n 'min_samples_leaf': 1,\n 'min_samples_split': 5,\n 'splitter': 'random'}\n"

In [45]:
dtc_pred = model.predict(xtest)
dtc_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [46]:
dtc_ac = accuracy_score(ytest,dtc_pred)
dtc_ac # witout cv ac = 0.8445

0.856

In [47]:
bias = model.score(xtrain,ytrain)
bias # without cv = 0.882125

0.87025

In [48]:
variance = model.score(xtest,ytest)
variance # without cv = 0.8445

0.856

# Let's apply Random Forest using GridSearchCv

In [49]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators = 100)

In [50]:
param_grid = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
#     'criterion': ['gini', 'entropy'],  # Splitting criterion for decision trees
#      'max_depth': [None, 10, 20, 30],  # Maximum depth of the trees
#     'min_samples_split': [2, 5, 10],  # Minimum samples required to split an internal node
#     'min_samples_leaf': [1, 2, 4],  # Minimum samples required to be at a leaf node
#     'max_features': ['auto', 'sqrt', 'log2'],  # Number of features to consider for the best split
#     'bootstrap': [True, False],  # Whether to bootstrap samples when building trees
#     'oob_score': [True, False],  # Whether to use out-of-bag samples for scoring
    'random_state': [0]  # Seed for random number generation
}

In [51]:
model = GridSearchCV(estimator=rfc,param_grid = param_grid, cv = 5)

In [52]:
model.fit(xtrain,ytrain)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'n_estimators': [100, 200, 300], 'random_state': [0]})

In [53]:
best_params = model.best_params_
best_params

{'n_estimators': 300, 'random_state': 0}

In [54]:
# Prediction
rfc_pred = model.predict(xtest)
rfc_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [55]:
# Accuracy
rf_ac = accuracy_score(ytest,rfc_pred)
rf_ac # .8645 for 100 estimators , 

0.8655

In [56]:
bias = model.score(xtrain,ytrain)
bias # 1.0 for 100 est

1.0

In [57]:
variance = model.score(xtest,ytest)
variance # .8645 for 100 est

0.8655

# Let's Apply Support Vector Machine Algorithm

In [58]:
from sklearn.svm import SVC
svc_classifier = SVC()

In [59]:
# # Define the parameter grid
# param_grid = {
#     'C': [0.1],  # Regularization parameter
#     'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],  # Type of kernel
#     'degree': [2, 3, 4],  # Degree for 'poly' kernel
#     'gamma': ['scale', 'auto'],  # Kernel coefficient for 'rbf', 'poly', 'sigmoid'
#     'coef0': [0.0, 0.5, 1.0],  # Independent term in kernel function for 'poly', 'sigmoid'
#     'shrinking': [True, False],  # Use the shrinking heuristic
#     'probability': [True, False],  # Enable probability estimates
#     'tol': [1e-3, 1e-4, 1e-5],  # Tolerance for stopping criterion
#     'cache_size': [100, 200, 300],  # Size of the kernel cache (in MB)
#     'class_weight': [None, 'balanced'],  # Set class weights
#     'verbose': [True, False],  # Enable verbose output
#     'max_iter': [1000, 2000, -1],  # Hard limit on iterations within solver
#     'decision_function_shape': ['ovo', 'ovr'],  # Decision function shape for multi-class problems
#     'break_ties': [True, False],  # Whether to break ties according to confidence values
#     'random_state': [0, 42]  # Controls random number generation for probability estimates
# }


In [60]:
# model = GridSearchCV(estimator = svc_classifier, param_grid=param_grid)

In [61]:
svc_classifier.fit(xtrain,ytrain)

SVC()

In [62]:
# Prediction
svc_pred = svc_classifier.predict(xtest)
svc_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [63]:
# Accuracy
svc_ac = accuracy_score(ytest,svc_pred)
svc_ac

0.7975

In [64]:
bias = svc_classifier.score(xtrain,ytrain)
bias

0.796

In [65]:
variance = svc_classifier.score(xtest,ytest)
variance

0.7975

In [66]:
cr = classification_report(ytest,svc_pred)
cr

'              precision    recall  f1-score   support\n\n           0       0.80      1.00      0.89      1595\n           1       0.00      0.00      0.00       405\n\n    accuracy                           0.80      2000\n   macro avg       0.40      0.50      0.44      2000\nweighted avg       0.64      0.80      0.71      2000\n'

# Let's Apply KNN Algorithm

In [67]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()

In [68]:
param_grid = {
    'n_neighbors': [3, 5, 7],  # Varying number of neighbors
    'weights': ['uniform', 'distance'],  # Types of weights
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  # Different algorithms
    'leaf_size': [20, 30, 40],  # Varying leaf size
    'p': [1, 2],  # Power parameter for Minkowski metric
    'metric': ['euclidean', 'manhattan'],  # Distance metric options
}

In [69]:
model = GridSearchCV(estimator=knn, param_grid = param_grid)

In [70]:
model.fit(xtrain,ytrain)

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'leaf_size': [20, 30, 40],
                         'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': [3, 5, 7], 'p': [1, 2],
                         'weights': ['uniform', 'distance']})

In [71]:
best_params = model.best_params_
best_params

{'algorithm': 'auto',
 'leaf_size': 20,
 'metric': 'euclidean',
 'n_neighbors': 7,
 'p': 1,
 'weights': 'uniform'}

In [72]:
# Prediction
knn_pred = model.predict(xtest)
knn_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [73]:
# Accuracy
knn_ac = accuracy_score(ytest,knn_pred)
knn_ac

0.779

In [74]:
# Bias
bias = model.score(xtrain,ytrain)
bias

0.808625

In [75]:
variance = model.score(xtest,ytest)
variance

0.779

In [76]:
cr = classification_report(ytest,knn_pred)
cr

'              precision    recall  f1-score   support\n\n           0       0.80      0.96      0.87      1595\n           1       0.31      0.07      0.12       405\n\n    accuracy                           0.78      2000\n   macro avg       0.55      0.52      0.49      2000\nweighted avg       0.70      0.78      0.72      2000\n'

# Let's apply Gradient boosting algorithm

- XGBoost

In [77]:
from xgboost import XGBClassifier
xgb = XGBClassifier()

In [78]:
# Fit the model on the training data
xgb.fit(xtrain, ytrain)

# Make predictions on the test data
y_pred = xgb.predict(xtest)





In [79]:
# Evaluate the model
xgb_ac = accuracy_score(ytest, y_pred)
xgb_ac

0.8525

In [80]:
print("Classification Report:")
xgb_cr = classification_report(ytest, y_pred)

Classification Report:


In [81]:
bias = xgb.score(xtrain,ytrain)
bias

0.957625

In [82]:
variance = xgb.score(xtest,ytest)
variance

0.8525

In [83]:
# # Define the parameter grid for XGBoost classifier
# param_grid = {
#     'n_estimators': [100, 200],  # Number of boosting rounds
#     'max_depth': [3, 4, 5],  # Maximum depth of a tree
#     'learning_rate': [0.1, 0.01, 0.001],  # Learning rate
#     'subsample': [0.8, 0.9, 1.0],  # Subsample ratio of the training instances
#     'colsample_bytree': [0.8, 0.9, 1.0],  # Subsample ratio of columns when constructing each tree
#     'gamma': [0, 0.1, 0.2]  # Mihttp://localhost:8888/notebooks/My%20Prog/Naresh_i%20_Technology/Resume%20Project/Customer%20Churn%20Analysis/Untitled.ipynb#nimum loss reduction required to make a further partition
# }
# '''

In [84]:
# model = GridSearchCV(estimator=xgb, param_grid=param_grid, cv = 5)

# Let's apply lightgbm 

In [85]:
import lightgbm as lgb
lgb_classifier = lgb.LGBMClassifier()

In [86]:
param_grid = {
    "boosting_type": ["gbdt", "rf", "dart"],
    "num_leaves": [20, 31, 40],
    "max_depth": [5, 6, 7],
    "learning_rate": [0.05, 0.1],
    # Add more hyperparameters and their values
}

In [87]:
model = GridSearchCV(estimator = lgb_classifier, param_grid= param_grid,cv = 5)

In [88]:
model.fit(xtrain,ytrain)

[LightGBM] [Info] Number of positive: 1306, number of negative: 5094
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000327 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 857
[LightGBM] [Info] Number of data points in the train set: 6400, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.204063 -> initscore=-1.361094
[LightGBM] [Info] Start training from score -1.361094
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 1306, number of negative: 5094
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000423 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 857
[LightGBM] [Info] Number of data points in the train set: 6400, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.204063 -> initscore=-1.361094
[LightGBM] [Info] Start training from score -1.361094
[LightGBM] [Warning] No further splits with positive gai

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 1305, number of negative: 5095
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000265 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 858
[LightGBM] [Info] Number of data points in the train set: 6400, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.203906 -> initscore=-1.362057
[LightGBM] [Info] Start training from score -1.362057
[LightGBM] [Warning] No further splits with positive gai

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 1305, number of negative: 5095
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000274 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 858
[LightGBM] [Info] Number of data points in the train set: 6400, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.203906 -> initscore=-1.362057
[LightGBM] [Info] Start training from score -1.362057
[LightGBM] [Warning] No further splits with positive gai

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 1305, number of negative: 5095
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000315 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 859
[LightGBM] [Info] Number of data points in the train set: 6400, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.20390

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 1306, number of negative: 5094
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000307 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 857
[LightGBM] [Info] Number of data points in the train set: 6400, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.204063 -> initscore=-1.361094
[LightGBM] [Info] Start training from score -1.361094
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Accuracy may be

GridSearchCV(cv=5, estimator=LGBMClassifier(),
             param_grid={'boosting_type': ['gbdt', 'rf', 'dart'],
                         'learning_rate': [0.05, 0.1], 'max_depth': [5, 6, 7],
                         'num_leaves': [20, 31, 40]})

In [89]:
best_params = model.best_params_
best_params

{'boosting_type': 'gbdt',
 'learning_rate': 0.05,
 'max_depth': 7,
 'num_leaves': 20}

In [90]:
# Prediction
lgb_pred = model.predict(xtest)
lgb_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [91]:
# Accuracy
lgb_ac = accuracy_score(ytest,lgb_pred)
lgb_ac

0.869

In [92]:
bias = model.score(xtrain,ytrain)
bias

0.882125

In [93]:
variance = model.score(xtest,ytest)
variance

0.869

In [94]:
cr = classification_report(ytest,lgb_pred)
cr

'              precision    recall  f1-score   support\n\n           0       0.89      0.95      0.92      1595\n           1       0.74      0.54      0.62       405\n\n    accuracy                           0.87      2000\n   macro avg       0.82      0.75      0.77      2000\nweighted avg       0.86      0.87      0.86      2000\n'

# Let's apply catboost

In [95]:
from catboost import CatBoostClassifier

In [96]:
cat_classifier = CatBoostClassifier()

In [97]:
cat_classifier.fit(xtrain,ytrain)

Learning rate set to 0.025035
0:	learn: 0.6730114	total: 144ms	remaining: 2m 23s
1:	learn: 0.6540997	total: 152ms	remaining: 1m 15s
2:	learn: 0.6343466	total: 159ms	remaining: 53s
3:	learn: 0.6184402	total: 167ms	remaining: 41.6s
4:	learn: 0.6032680	total: 175ms	remaining: 34.8s
5:	learn: 0.5878109	total: 183ms	remaining: 30.3s
6:	learn: 0.5758736	total: 192ms	remaining: 27.3s
7:	learn: 0.5625693	total: 201ms	remaining: 24.9s
8:	learn: 0.5492017	total: 209ms	remaining: 23.1s
9:	learn: 0.5381937	total: 221ms	remaining: 21.9s
10:	learn: 0.5272795	total: 228ms	remaining: 20.5s
11:	learn: 0.5178577	total: 238ms	remaining: 19.6s
12:	learn: 0.5082159	total: 244ms	remaining: 18.6s
13:	learn: 0.4990721	total: 253ms	remaining: 17.8s
14:	learn: 0.4913408	total: 260ms	remaining: 17.1s
15:	learn: 0.4835522	total: 270ms	remaining: 16.6s
16:	learn: 0.4778423	total: 277ms	remaining: 16s
17:	learn: 0.4714120	total: 286ms	remaining: 15.6s
18:	learn: 0.4656854	total: 293ms	remaining: 15.1s
19:	learn: 0.

186:	learn: 0.3145899	total: 1.6s	remaining: 6.97s
187:	learn: 0.3144011	total: 1.61s	remaining: 6.95s
188:	learn: 0.3142586	total: 1.62s	remaining: 6.93s
189:	learn: 0.3140891	total: 1.62s	remaining: 6.92s
190:	learn: 0.3139168	total: 1.63s	remaining: 6.91s
191:	learn: 0.3137835	total: 1.64s	remaining: 6.9s
192:	learn: 0.3136108	total: 1.65s	remaining: 6.89s
193:	learn: 0.3133949	total: 1.66s	remaining: 6.88s
194:	learn: 0.3132820	total: 1.66s	remaining: 6.87s
195:	learn: 0.3131479	total: 1.67s	remaining: 6.85s
196:	learn: 0.3129952	total: 1.68s	remaining: 6.83s
197:	learn: 0.3128488	total: 1.68s	remaining: 6.82s
198:	learn: 0.3127065	total: 1.69s	remaining: 6.8s
199:	learn: 0.3125918	total: 1.7s	remaining: 6.79s
200:	learn: 0.3124922	total: 1.7s	remaining: 6.77s
201:	learn: 0.3122801	total: 1.71s	remaining: 6.76s
202:	learn: 0.3121879	total: 1.72s	remaining: 6.74s
203:	learn: 0.3120249	total: 1.72s	remaining: 6.73s
204:	learn: 0.3118224	total: 1.73s	remaining: 6.71s
205:	learn: 0.311

354:	learn: 0.2941205	total: 2.82s	remaining: 5.12s
355:	learn: 0.2939423	total: 2.83s	remaining: 5.11s
356:	learn: 0.2938546	total: 2.83s	remaining: 5.1s
357:	learn: 0.2937730	total: 2.84s	remaining: 5.09s
358:	learn: 0.2936407	total: 2.85s	remaining: 5.09s
359:	learn: 0.2935233	total: 2.86s	remaining: 5.08s
360:	learn: 0.2934106	total: 2.87s	remaining: 5.07s
361:	learn: 0.2933020	total: 2.87s	remaining: 5.06s
362:	learn: 0.2932144	total: 2.88s	remaining: 5.05s
363:	learn: 0.2930522	total: 2.89s	remaining: 5.05s
364:	learn: 0.2929733	total: 2.9s	remaining: 5.04s
365:	learn: 0.2928512	total: 2.9s	remaining: 5.03s
366:	learn: 0.2927542	total: 2.91s	remaining: 5.02s
367:	learn: 0.2926231	total: 2.92s	remaining: 5.01s
368:	learn: 0.2925426	total: 2.92s	remaining: 5s
369:	learn: 0.2924242	total: 2.93s	remaining: 4.99s
370:	learn: 0.2923288	total: 2.94s	remaining: 4.98s
371:	learn: 0.2922256	total: 2.95s	remaining: 4.97s
372:	learn: 0.2921358	total: 2.96s	remaining: 4.97s
373:	learn: 0.2920

514:	learn: 0.2764910	total: 4.05s	remaining: 3.81s
515:	learn: 0.2763752	total: 4.05s	remaining: 3.8s
516:	learn: 0.2762614	total: 4.06s	remaining: 3.79s
517:	learn: 0.2761841	total: 4.07s	remaining: 3.79s
518:	learn: 0.2760769	total: 4.08s	remaining: 3.78s
519:	learn: 0.2759634	total: 4.08s	remaining: 3.77s
520:	learn: 0.2758498	total: 4.09s	remaining: 3.76s
521:	learn: 0.2757699	total: 4.1s	remaining: 3.75s
522:	learn: 0.2756646	total: 4.11s	remaining: 3.74s
523:	learn: 0.2755922	total: 4.11s	remaining: 3.74s
524:	learn: 0.2754815	total: 4.12s	remaining: 3.73s
525:	learn: 0.2754099	total: 4.13s	remaining: 3.72s
526:	learn: 0.2753363	total: 4.13s	remaining: 3.71s
527:	learn: 0.2752074	total: 4.14s	remaining: 3.7s
528:	learn: 0.2750677	total: 4.15s	remaining: 3.69s
529:	learn: 0.2749638	total: 4.16s	remaining: 3.69s
530:	learn: 0.2748874	total: 4.16s	remaining: 3.68s
531:	learn: 0.2747597	total: 4.17s	remaining: 3.67s
532:	learn: 0.2746600	total: 4.18s	remaining: 3.66s
533:	learn: 0.2

685:	learn: 0.2601333	total: 5.3s	remaining: 2.43s
686:	learn: 0.2600469	total: 5.31s	remaining: 2.42s
687:	learn: 0.2599857	total: 5.32s	remaining: 2.41s
688:	learn: 0.2599362	total: 5.33s	remaining: 2.4s
689:	learn: 0.2598736	total: 5.33s	remaining: 2.4s
690:	learn: 0.2597427	total: 5.34s	remaining: 2.39s
691:	learn: 0.2596472	total: 5.35s	remaining: 2.38s
692:	learn: 0.2595722	total: 5.36s	remaining: 2.37s
693:	learn: 0.2595084	total: 5.37s	remaining: 2.37s
694:	learn: 0.2594443	total: 5.37s	remaining: 2.36s
695:	learn: 0.2593578	total: 5.38s	remaining: 2.35s
696:	learn: 0.2592227	total: 5.39s	remaining: 2.34s
697:	learn: 0.2591373	total: 5.39s	remaining: 2.33s
698:	learn: 0.2590742	total: 5.4s	remaining: 2.33s
699:	learn: 0.2589806	total: 5.41s	remaining: 2.32s
700:	learn: 0.2588940	total: 5.42s	remaining: 2.31s
701:	learn: 0.2588150	total: 5.42s	remaining: 2.3s
702:	learn: 0.2587376	total: 5.43s	remaining: 2.29s
703:	learn: 0.2586656	total: 5.44s	remaining: 2.29s
704:	learn: 0.258

853:	learn: 0.2461271	total: 6.54s	remaining: 1.12s
854:	learn: 0.2460661	total: 6.54s	remaining: 1.11s
855:	learn: 0.2459769	total: 6.55s	remaining: 1.1s
856:	learn: 0.2458529	total: 6.56s	remaining: 1.09s
857:	learn: 0.2457939	total: 6.57s	remaining: 1.09s
858:	learn: 0.2456958	total: 6.57s	remaining: 1.08s
859:	learn: 0.2456168	total: 6.58s	remaining: 1.07s
860:	learn: 0.2455202	total: 6.59s	remaining: 1.06s
861:	learn: 0.2454571	total: 6.6s	remaining: 1.06s
862:	learn: 0.2453947	total: 6.6s	remaining: 1.05s
863:	learn: 0.2452874	total: 6.61s	remaining: 1.04s
864:	learn: 0.2452155	total: 6.62s	remaining: 1.03s
865:	learn: 0.2451512	total: 6.63s	remaining: 1.02s
866:	learn: 0.2451034	total: 6.63s	remaining: 1.02s
867:	learn: 0.2450045	total: 6.64s	remaining: 1.01s
868:	learn: 0.2449197	total: 6.65s	remaining: 1s
869:	learn: 0.2448505	total: 6.66s	remaining: 995ms
870:	learn: 0.2447801	total: 6.66s	remaining: 987ms
871:	learn: 0.2447272	total: 6.67s	remaining: 979ms
872:	learn: 0.2446

In [98]:
# catboost_param_grid = {
#     'iterations': [100, 200, 300],
#     'learning_rate': [0.1, 0.05, 0.01],
#     'depth': [4, 6, 8],
#     'l2_leaf_reg': [3.0, 5.0, 7.0],
#     'model_size_reg': [None, 0.1, 0.5],
#     'rsm': [0.7, 0.8, 1.0],
#     'loss_function': ['Logloss', 'CrossEntropy'],
#     'border_count': [254, 128],
#     # ... (additional parameters and their respective values)
#     'used_ram_limit': ['1.2gb', '2.4gb'],
#     'eval_fraction': [0.1, 0.2, 0.3],
#     'bagging_temperature': [0.5, 1.0, 1.5],
#     'bootstrap_type': ['Bayesian', 'Bernoulli', 'Poisson'],
#     # ... (more parameters and their respective values)
# }


In [99]:
# model = GridSearchCV(estimator = cat_classifier,param_grid = catboost_param_grid,cv = 5)

In [100]:
# Prediction
cat_pred = cat_classifier.predict(xtest)
cat_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [101]:
# Accuracy
cat_ac = accuracy_score(ytest,cat_pred)
cat_ac

0.8655

In [102]:
bias = cat_classifier.score(xtrain,ytrain)
bias

0.9115

In [103]:
variance = cat_classifier.score(xtest,ytest)
variance

0.8655

In [104]:
cr = classification_report(ytest,cat_pred)
cr

'              precision    recall  f1-score   support\n\n           0       0.89      0.95      0.92      1595\n           1       0.73      0.53      0.62       405\n\n    accuracy                           0.87      2000\n   macro avg       0.81      0.74      0.77      2000\nweighted avg       0.86      0.87      0.86      2000\n'

# Time to find the best model for our dataset

In [111]:
algo = ['Logistic Regression','Bernouli Naive Bayes','Gaussian Naive Bayes','MultiNomial Naive Bayes','Decision Trees','Random Forest',
        'Support Vector Machines (SVM)','k-Nearest Neighbors (k-NN)','XGBoost', 'LightGBM', 'CatBoost']

In [112]:
accuracy = [logit_ac,bnb_ac,gnb_ac,mnb_ac,dtc_ac,rf_ac,svc_ac,knn_ac,xgb_ac,lgb_ac,cat_ac]

In [113]:
print(len(algo))
print(len(accuracy))

11
11


In [114]:
for model, acc in zip(algo, accuracy):
    print(f"{model} model: Accuracy - {acc}")

Logistic Regression model: Accuracy - 0.789
Bernouli Naive Bayes model: Accuracy - 0.786
Gaussian Naive Bayes model: Accuracy - 0.7845
MultiNomial Naive Bayes model: Accuracy - 0.544
Decision Trees model: Accuracy - 0.856
Random Forest model: Accuracy - 0.8655
Support Vector Machines (SVM) model: Accuracy - 0.7975
k-Nearest Neighbors (k-NN) model: Accuracy - 0.779
XGBoost model: Accuracy - 0.8525
LightGBM model: Accuracy - 0.869
CatBoost model: Accuracy - 0.8655


In [108]:
# Alternate of showing
# algo = ['Logistic Regression', 'Bernoulli Naive Bayes', 'Gaussian Naive Bayes', 'MultiNomial Naive Bayes', 'Decision Trees', 'Random Forest',
#         'Support Vector Machines (SVM)', 'k-Nearest Neighbors (k-NN)', 'XGBoost', 'LightGBM', 'CatBoost']
# accuracy = [logit_ac, bnb_ac, gnb_ac, mnb_ac, dtc_ac, rf_ac, svc_ac, knn_ac, xgb_ac, lgb_ac, cat_ac]

# for i in range(len(algo)):
#     print(f"{algo[i]}: Accuracy - {accuracy[i]}")